<a href="https://colab.research.google.com/github/Moksha-nagraj/Marvel_tasks_lvl2/blob/main/Copy_of_ID3_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Loading the dataset
train_data_m = pd.read_csv("/content/PlayTennis.csv")

def calc_total_entropy(train_data, label, class_list):
    total_row = train_data.shape[0]  # total size of the dataset
    total_entr = 0

    for c in class_list:  # for each class in the label
        total_class_count = train_data[train_data[label] == c].shape[0]  # number of the class
        if total_class_count == 0:
            continue
        total_class_entr = - (total_class_count / total_row) * np.log2(total_class_count / total_row)  # entropy of the class
        total_entr += total_class_entr  # adding the class entropy to the total entropy of the dataset

    return total_entr

def calc_entropy(feature_value_data, label, class_list):
    class_count = feature_value_data.shape[0]
    entropy = 0

    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0]  # row count of class c
        if label_class_count == 0:
            continue
        probability_class = label_class_count / class_count  # probability of the class
        entropy_class = - probability_class * np.log2(probability_class)  # entropy
        entropy += entropy_class
    return entropy

def calc_info_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique()  # unique values of the feature
    total_row = train_data.shape[0]
    feature_info = 0.0

    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value]  # filtering rows with that feature_value
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list)  # calculating entropy for the feature value
        feature_value_probability = feature_value_count / total_row
        feature_info += feature_value_probability * feature_value_entropy  # calculating information of the feature value

    total_entropy = calc_total_entropy(train_data, label, class_list)
    info_gain = total_entropy - feature_info
    print(f"Info gain for {feature_name}: {info_gain}")
    return info_gain

def find_most_informative_feature(train_data, label, class_list):
    feature_list = train_data.columns.drop(label)  # finding the feature names in the dataset
    max_info_gain = -1
    max_info_feature = None

    for feature in feature_list:  # for each feature in the dataset
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain:  # selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature

    print(f"Most informative feature: {max_info_feature}")
    return max_info_feature

def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False)  # dictionary of the count of unique feature value
    tree = {}  # sub tree or node

    for feature_value, count in feature_value_count_dict.items():
        feature_value_data = train_data[train_data[feature_name] == feature_value]  # dataset with only feature_name = feature_value

        assigned_to_node = False  # flag for tracking feature_value is pure class or not
        for c in class_list:  # for each class
            class_count = feature_value_data[feature_value_data[label] == c].shape[0]  # count of class c

            if class_count == count:  # count of (feature_value = count) of class (pure class)
                tree[feature_value] = c  # adding node to the tree
                assigned_to_node = True
        if not assigned_to_node:  # not pure class
            tree[feature_value] = "?"  # as feature_value is not a pure class, it should be expanded further,
                                      # so the branch is marking with ?

    print(f"Generated sub-tree for {feature_name}: {tree}")
    return tree

def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0:  # if dataset is not empty after updating
        max_info_feature = find_most_informative_feature(train_data, label, class_list)  # most informative feature
        if max_info_feature is None:
            return
        tree = generate_sub_tree(max_info_feature, train_data, label, class_list)  # getting tree node and updated dataset

        if prev_feature_value is not None:  # add to intermediate node of the tree
            root[prev_feature_value] = {max_info_feature: tree}
        else:  # add to root of the tree
            root[max_info_feature] = tree

        for feature_value, sub_tree in tree.items():
            if sub_tree == "?":  # if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == feature_value]  # using the updated dataset
                make_tree(tree, feature_value, feature_value_data, label, class_list)  # recursive call with updated dataset

def id3(train_data_m, label):
    train_data = train_data_m.copy()  # getting a copy of the dataset
    tree = {}  # tree which will be updated
    class_list = train_data[label].unique()  # getting unique classes of the label
    make_tree(tree, None, train_data, label, class_list)  # start calling recursion
    print("Generated tree:", tree)
    return tree

tree = id3(train_data_m, 'Play Tennis')

def predict(tree, instance):
    if not isinstance(tree, dict):  # if it is leaf node
        return tree  # return the value
    elif not tree:  # check if the dictionary is empty
        print("Empty tree encountered!")
        return None
    else:
        root_node = next(iter(tree))  # getting first key/feature name of the dictionary
        feature_value = instance[root_node]  # value of the feature
        if feature_value in tree[root_node]:  # checking the feature value in current tree node
            return predict(tree[root_node][feature_value], instance)  # go to next feature
        else:
            return None

def evaluate(tree, test_data_m, label):
    correct_predict = 0
    wrong_predict = 0
    for index, row in test_data_m.iterrows():  # for each row in the dataset
        result = predict(tree, test_data_m.iloc[index])  # predict the row
        if result == test_data_m[label].iloc[index]:  # predicted value and expected value is same or not
            correct_predict += 1  # increase correct count
        else:
            wrong_predict += 1  # increase incorrect count
    accuracy = correct_predict / (correct_predict + wrong_predict)  # calculating accuracy
    return accuracy

test_data_m = pd.read_csv("/content/PlayTennis.csv")  # importing test dataset into dataframe

accuracy = evaluate(tree, test_data_m, 'Play Tennis')  # evaluating the test dataset
print(f"Accuracy: {accuracy}")


Info gain for Outlook: 0.24674981977443933
Info gain for Temperature: 0.02922256565895487
Info gain for Humidity: 0.15183550136234159
Info gain for Wind: 0.04812703040826949
Most informative feature: Outlook
Generated sub-tree for Outlook: {'Sunny': '?', 'Overcast': 'Yes', 'Rain': '?'}
Info gain for Outlook: 0.0
Info gain for Temperature: 0.5709505944546686
Info gain for Humidity: 0.9709505944546686
Info gain for Wind: 0.01997309402197489
Most informative feature: Humidity
Generated sub-tree for Humidity: {'High': 'No', 'Normal': 'Yes'}
Info gain for Outlook: 0.0
Info gain for Temperature: 0.01997309402197489
Info gain for Humidity: 0.01997309402197489
Info gain for Wind: 0.9709505944546686
Most informative feature: Wind
Generated sub-tree for Wind: {'Weak': 'Yes', 'Strong': 'No'}
Generated tree: {'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}, 'Overcast': 'Yes', 'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}
Accuracy: 1.0
